# Using VO Tables from HEASARC 

In this notebook, we show how to list, fetch and look at VO tables from the HEASARC archive and also how to create your own VO Table, which can be used to cross-correlate with other VO Tables.  

*  [1. Retrieving](#retrieve) a catalog as a VO Table
*  [2. Creating](#create) a VOTable from scratch
*  [3. Uploading](#upload) our VOTable file for cross-correlation

In [1]:
## The main HTTP request tool we will use:
import requests, io, numpy
from astropy import table as aptable

<a id="retrieve"></a>

# 1. Retrieving a catalog as a VO Table

We can use __[Browse](https://heasarc.gsfc.nasa.gov/cgi-bin/W3Browse/w3browse.pl)__ to get any HEASARC catalog.  Browse can be used to retrieve tables in a number of formats, one of which is a VOTable in XML as defined by the __[IVOA reference](http://www.ivoa.net/documents/VOTable/)__.  

The requests package of python is a generic interface to web services.  The requests.get() method formats the given parameters for you and returns the results of the HTTP request in a useful object.  In this case, the result is a string containing the VOTable XML.  The standard astropy.table package can read these into the standard Table objec.

This fetches the entire rosmaster catalog, so give it a few seconds:

In [2]:
params = {'name': 'rosmaster'}
r = requests.get('http://heasarc.gsfc.nasa.gov/cgi-bin/W3Browse/getvotable.pl', params=params)
#r.text

The astropy.table package can read the VOTable's XML and create a usual Table object (with some warnings that we can ignore):

In [3]:
table=aptable.Table.read(io.BytesIO(r.content))

In [4]:
print(type(table))
table

<class 'astropy.table.table.Table'>


col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28
,deg,deg,deg,deg,,,,s,s,,MJD,MJD,,,,,,,,,,,,deg,d,d,
bytes16,float64,float64,float64,float64,bytes6,bytes2,bytes4,int32,int32,bytes10,float64,float64,bytes20,bytes16,bytes14,int32,bytes4,int16,int16,bytes120,int32,int16,int32,int16,int32,int32,int16
RH202299N00,49.3200,-85.5400,299.8517,-30.6815,HRI,N,MPE,36146,70000,RDF 4_2,50174.4961921296,50187.9867476852,RE J0317-853,BURLEIGH,MATTHEW,202299,n00,2,2,RE J0317-853,24302,6,7128,307,2163,2177,2900
RH202299A01,49.3200,-85.5400,299.8517,-30.6815,HRI,N,MPE,43683,70000,RDF 3_6,50324.7425347222,50377.4775925926,RE J0317-853,BURLEIGH,MATTHEW,202299,a01,2,2,RE J0317-853,24301,6,7128,162,2316,2369,2900
RP200594N00,151.8800,-85.0800,299.4192,-23.3707,PSPCB,N,MPE,4231,5000,RDF 3_4,48682.1389583333,48690.9833333333,HD 89499,FLEMING,THOMAS,200594,n00,2,2,HD 89499,17171,2,3532,66,648,657,2900
RH201328N00,107.9100,-84.4700,296.7740,-26.5072,HRI,N,MPE,2332,2000,RDF 3_4,48880.8380208333,48881.7807291667,HD 60102 AB,ZINNECKER,HANS,201328,n00,2,2,HD 60102 AB,22560,3,4268,215,850,851,2900
RH400876A01,193.9400,-83.3700,303.0651,-20.4994,HRI,N,GSFC,19075,20000,RDF 4_2,50702.4235300926,50735.1725578704,2EG J1248-8306 POS 3,HALPERN,JULES,400876,a01,4,2,2EG J1248-8306 POS 3,25116,7,2460,299,2699,2733,9999
RH400876N00,193.9400,-83.3700,303.0651,-20.4994,HRI,N,GSFC,1059,20000,RDF 4_0,50548.2425,50548.255474537,2EG J1248-8306 POS 3,HALPERN,JULES,400876,n00,4,2,2EG J1248-8306 POS 3,19808,7,2460,85,2543,2543,9999
RH400877A02,190.1700,-83.3600,302.6001,-20.4952,HRI,N,GSFC,19384,20000,RDF 4_2,50893.2484606481,50895.1389699074,2EG J1248-8306 POS 4,HALPERN,JULES,400877,a02,4,2,2EG J1248-8306 POS 4,24927,7,2460,90,2893,2895,9999
RH400877A01,190.1700,-83.3600,302.6001,-20.4952,HRI,N,GSFC,2363,20000,RDF 4_2,50735.5647916667,50735.5741203704,2EG J1248-8306 POS 4,HALPERN,JULES,400877,a01,4,2,2EG J1248-8306 POS 4,25115,7,2460,260,2733,2733,9999


The current astroquery.heasarc package has limited functionality, but it will return astropy Tables from object queries to HEASARC catalogs.  (It does not currently use the VO interface.)  For example:

In [5]:
from astroquery.heasarc import Heasarc
heasarc=Heasarc()
heasarc.query_object('3c273',"rosmaster")

SEQ_ID,INSTRUMENT,EXPOSURE,RA,DEC,NAME,SEARCH_OFFSET_
,,S,DEGREE,DEGREE,,
bytes11,bytes10,int32,float64,float64,bytes20,bytes15
RH701576N00,HRI,68154,187.28,2.05,3C 273,0.192 (3C273)
RP600242A01,PSPCB,24822,186.93,1.6,GIOVANELLI-HAYNES CL,34.236 (3C273)
RH700234N00,HRI,17230,187.28,2.05,3C 273,0.192 (3C273)
RP700191N00,PSPCB,6140,187.27,2.05,3C273,0.495 (3C273)
RH702682N00,HRI,4896,187.27,2.05,3C 273,0.495 (3C273)
RH702681N00,HRI,4416,187.27,2.05,3C 273,0.495 (3C273)
RH702684N00,HRI,4105,187.27,2.05,3C 273,0.495 (3C273)
RH702680N00,HRI,4063,187.27,2.05,3C 273,0.495 (3C273)


<a id="create"></a>
# 2. Creating a VO Table from scratch

Alternatively, we can create a table from scratch using an array of values named whatever we want.  

There are several ways of doing this, and there are a few object layers here, which can be confusing.  There are standard astropy Table objects, there are votable Table objects, and there are votable VOTableFile objects that might contain multiple votable Tables. Though some things can be done with generic astropy Tables, other VO operations can only be done with VO Tables or VOTableFile objects.  

This is easiest to see with an example.  

In [6]:
##  Create a table with only two columns starting from an astropy Table:  
myaptable=aptable.Table(
    numpy.array([
            [19.0186,       46.7304],
            [20.2887,       40.4703],
            [125.886,       21.3377],
            [136.002,       21.9679],
            [141.057,       40.6372],
            [146.700,       22.0116],
            [148.785,       14.2922],
            [149.751,       17.8168],
            [175.039,       15.3270],
            [191.542,       30.7317],
            [194.913,       28.8959],
            [199.026,       41.5011],
            [206.577,       43.8511],
            [209.963,       38.1821],
            [213.556,       15.6214],
            [219.967,       42.7421],
            [226.693,       12.8502],
            [237.489,       20.8057],
            [241.519,       20.8014],
            [317.088,       18.2002],
            [329.235,       6.64845],
            [333.830,       37.3012] ]), 
    names=["RA","DEC"])
print(type(myaptable))
print(myaptable)


<class 'astropy.table.table.Table'>
   RA     DEC  
------- -------
19.0186 46.7304
20.2887 40.4703
125.886 21.3377
136.002 21.9679
141.057 40.6372
  146.7 22.0116
148.785 14.2922
149.751 17.8168
175.039  15.327
191.542 30.7317
194.913 28.8959
199.026 41.5011
206.577 43.8511
209.963 38.1821
213.556 15.6214
219.967 42.7421
226.693 12.8502
237.489 20.8057
241.519 20.8014
317.088 18.2002
329.235 6.64845
 333.83 37.3012


In [7]:
from astropy.io import votable as apvot

## Then convert this to a VOTableFile object.
myvotablefile = apvot.from_table(myaptable)
print(type(myvotablefile))
## Which contains a nested set of RESOURCES
for r in myvotablefile.resources:
    ## And tables (in this case, only one of each)
    mytable=r
    for t in r.tables:
        print(t)


<class 'astropy.io.votable.tree.VOTableFile'>
   RA     DEC  
------- -------
19.0186 46.7304
20.2887 40.4703
125.886 21.3377
136.002 21.9679
141.057 40.6372
  146.7 22.0116
148.785 14.2922
149.751 17.8168
175.039  15.327
191.542 30.7317
194.913 28.8959
199.026 41.5011
206.577 43.8511
209.963 38.1821
213.556 15.6214
219.967 42.7421
226.693 12.8502
237.489 20.8057
241.519 20.8014
317.088 18.2002
329.235 6.64845
 333.83 37.3012


<a id="upload"></a>

# 3. Uploading our VOTable file to cross-correlate

The reason we might want not just an astropy Table object but a votable Table object is that we might want to cross-correlate our list of objects with another catalog.  In this case, we would use the __[Table Access Protocol](http://www.ivoa.net/documents/TAP/)__ discussed more extensively in a different notebook.  

Here, we simply show an example of how to hand our VOTable object to the HEASARC TAP service.

We start by writing the votablefile object to a "file-like" object (using BytesIO) that requests will be able to upload (alternatively we could write it to disk, but why?):

In [14]:
## Alternatively, write it to a file:
apvot.writeto(myvotablefile, "new_votable.xml")
#files={'uplt':open('new_votable.xml', 'rb')}

For more information on how to construct the TAP query, see its dedicated notebook.  For our purposes here, this is simply to demonstrate that we can use our VOTable to cross-correlate with, e.g., the CfA Redshift Catalog (zcat).  In other words, the follow retrieves every entry in the zcat that is within a radius of 0.1deg of each of our 30 sources, returning the zcat entries for RA, DEC, and Radial_Velocity.  

## BROKEN. 404 from what? The url?

In [18]:
## astroquery has a library for TAP queries:
from astroquery.utils.tap.core import TapPlus

## For now, you have to look up the URL for this TAP service. To be fixed.
service = TapPlus(url='https://heasarc.gsfc.nasa.gov/xamin/vo/tap/sync')
query="""
    SELECT cat.ra, cat.dec, Radial_Velocity 
    FROM zcat cat, tap_upload.mysources mt 
    WHERE
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,0.1))=1
    and Radial_Velocity > 0
    ORDER by cat.ra"""

## Cannot seem to do it with an IO stream in memory
#job = service.launch_job(query,upload_resource=vot_obj,upload_table_name="mysources")
job = service.launch_job(query,upload_resource="new_votable.xml",upload_table_name="mysources",verbose=True)

results = job.get_results()
print(results)

Created TAP+ (v1.0.1) - Connection:
	Host: heasarc.gsfc.nasa.gov
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Launched query: '
    SELECT  TOP 2000 cat.ra, cat.dec, Radial_Velocity 
    FROM zcat cat, tap_upload.mysources mt 
    WHERE
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,0.1))=1
    and Radial_Velocity > 0
    ORDER by cat.ra'
404 Not Found
[('Date', 'Wed, 23 May 2018 18:10:00 GMT'), ('Content-Type', 'text/html;charset=UTF-8'), ('Content-Length', '292'), ('Set-Cookie', 'JSESSIONID=E74DD6D6074301794786582542EC2114;path=/xamin;HttpOnly'), ('Strict-Transport-Security', 'max-age=31536000; includeSubDomains')]
ERROR: 404: Not Found


Exception: Not Found

## BROKEN: this is the problem I sent to TomM about xamin not working. 

(Don't get this now, but the above 404. This is what I used to get.)

In [21]:
## In memory only, use an IO stream. 
vot_obj=io.BytesIO()
apvot.writeto(myvotablefile,vot_obj)
vot_obj.seek(0) # Reset seek to beginning of buffer
## 'uplt' is what we'll call it (for 'upload table') 
##   in the requests parameters below, or what you will:
files={'uplt':vot_obj}

(This takes a good 20 seconds. If you need to rerun this cell, rerun the one above that defines the upload.  The IO object needs to be reset.) 

In [22]:
## These parameters are defined in the TAP standard.  
##  The string 'uplt' is what we set above as the name
##  of the parameter containing this data, and
##  the string 'mysources' is how we refer to it in 
##  the ADQL query string:
cc_params={
    'lang': 'ADQL', 
    'request': 'doQuery',
    'upload':'mysources,param:uplt'
    }
cc_params["query"]="""
    SELECT cat.ra, cat.dec, Radial_Velocity 
    FROM zcat cat, tap_upload.mysources mt 
    WHERE
    contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',mt.ra,mt.dec,0.1))=1
    and Radial_Velocity > 0
    ORDER by cat.ra"""
r = requests.post('https://heasarc.gsfc.nasa.gov/xamin/vo/tap/sync',data=cc_params,stream=True,files=files)
print(r.content)
mytable=aptable.Table.read(io.BytesIO(r.content))
mytable

b'<?xml version="1.0"?>\n<VOTABLE xsi:noNamespaceSchemaLocation="http://www.ivoa.net/VOTable/VOTable/v1.3" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="1.3">\n <RESOURCE type="results">\n  <INFO name="QUERY_STATUS" value="ERROR">Error[s] detected in query preprocessing</INFO>\n  <INFO name="Failure:Security" value="Invalid SQL: SELECT cat.ra, cat.dec, Radial_Velocity from zcat cat, tap_upload_1527099066626_mysources mt &#xA;    WHERE&#xA;    contains(point(\'ICRS\',cat.ra,cat.dec),circle(\'ICRS\',mt.ra,mt.dec,0.1))=1&#xA;    and Radial_Velocity &gt; 0&#xA;    ORDER by cat.ra,&#xA;    SELECT cat.ra, cat.dec, Radial_Velocity from zcat cat, tap_upload_1527099066626_mysources mt &#xA;    WHERE&#xA;    contains(point(\'ICRS\',cat.ra,cat.dec),circle(\'ICRS\',mt.ra,mt.dec,0.1))=1&#xA;    and Radial_Velocity &gt; 0&#xA;    ORDER by cat.ra"/>\n </RESOURCE>\n <RESOURCE utype="adhoc:service" type="meta">\n  <PARAM name="standardID" value="ivo://ivoa.net/std/DataLink#links-1.0" a

ValueError: No table found